In [13]:
import glob, os
import pandas as pd
import csv
import dateutil.parser
%matplotlib inline
import matplotlib
import datetime
import calendar
import calendar
import numpy as np
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
from geopy.geocoders import Nominatim

In [14]:
import sys  

reload(sys)  
sys.setdefaultencoding('utf8')

# Station data

Lets clean the mta station entrance data

In [15]:
station_df= pd.read_csv('geo/StationEntrances.csv')                      

In [16]:
station_df.head()

,Division,Line,Station_Name,Station_Latitude,Station_Longitude,Route_1,Route_2,Route_3,Route_4,Route_5,...,Staffing,Staff_Hours,ADA,ADA_Notes,Free_Crossover,North_South_Street,East_West_Street,Corner,Latitude,Longitude
0,BMT,Astoria,Ditmars Blvd,40.775036,-73.912034,N,Q,NaN,NaN,NaN,...,FULL,NaN,False,NaN,True,31st St,23rd Ave,NW,40.775149,-73.912074
1,BMT,Astoria,Ditmars Blvd,40.775036,-73.912034,N,Q,NaN,NaN,NaN,...,FULL,NaN,False,NaN,True,31st St,23rd Ave,NE,40.774810,-73.912151
2,BMT,Astoria,Ditmars Blvd,40.775036,-73.912034,N,Q,NaN,NaN,NaN,...,FULL,NaN,False,NaN,True,31st St,23rd Ave,NE,40.775025,-73.911891
3,BMT,Astoria,Ditmars Blvd,40.775036,-73.912034,N,Q,NaN,NaN,NaN,...,FULL,NaN,False,NaN,True,31st St,23rd Ave,NW,40.774938,-73.912337
4,BMT,Astoria,Astoria Blvd-Hoyt Av,40.770258,-73.917843,N,Q,NaN,NaN,NaN,...,FULL,NaN,False,NaN,True,31st St,Hoyt Ave South,SW,40.770313,-73.917978


In [17]:
stations_grouped=station_df.groupby('Station_Name').agg({'Station_Latitude':min,'Station_Longitude':min}) 
stations_grouped=stations_grouped.reset_index()
stations_grouped=stations_grouped.rename(columns={'Station_Latitude':'Latitude','Station_Longitude':'Longitude'})
stations_grouped.head()

,Station_Name,Latitude,Longitude
0,103rd St,40.749865,-73.968379
1,104th St-102nd St,40.695178,-73.844330
2,104th St-Oxford Av,40.681711,-73.837683
3,110th St,40.795020,-73.944250
4,110th St-Central Park North,40.799075,-73.951822


In [18]:
stations_grouped.to_csv('geo/stations_geo.csv',index=False)

# Company data

Lets find the list of companies in ny

In [19]:
company_df= pd.read_csv('geo/us_companies.csv')
company_df.head()

,company_name_id,company_name,url,year_founded,city,state,country,zip_code,full_time_employees,company_type,...,business_model,social_impact,description,description_short,source_count,data_types,example_uses,data_impacts,financial_info,last_updated
0,3-round-stones-inc,"3 Round Stones, Inc.",http://3RoundStones.com,2010,Washington,DC,us,20004,1-10,Private,...,"Business to Business, Business to Consumer",NaN,3 Round Stones produces a platform for publish...,Our Open Source platform is used by the Fortun...,NaN,NaN,NaN,[],"3 Round Stones is a profitable, self-funded, w...",2014-11-12 14:44:25.969871
1,48-factoring-inc,48 Factoring Inc.,https://www.48factoring.com,2014,Philadelphia,PA,us,19087,51-200,Private,...,Business to Business,Small Business Owners,The company mission is to provide finance to s...,48 Factoring Inc. is one of the best financial...,11-50,Business,NaN,"[u'Cost efficiency', u'Job growth', u'Revenue ...",NaN,2015-02-12 16:47:20.164268
2,5psolutions,5PSolutions,www.5psolutions.com,2007,Fairfax,VA,us,22003,1-10,Private,...,"Business to Business, Business to Consumer, Bu...",NaN,"At 5PSolutions, we wish to make all basic info...",5PSolutions are artisans of mobile platforms.,NaN,NaN,NaN,[],NaN,2014-11-12 15:09:35.485949
3,abt-associates,Abt Associates,abtassoc.com,1965,Cambridge,MA,us,2138,"1,001-5,000",Private,...,NaN,NaN,"Abt Associates is a mission-driven, internatio...","Abt Associates is a mission-driven, global lea...",101+,NaN,NaN,[],Employee-owned company. $552M/year.,2014-09-11 17:23:21.390289
4,accela,Accela,http://www.accela.com,1999,San Ramon,CA,us,94583,"501-1,000",Private,...,Business to Consumer,"Citizen engagement and participation, Good gov...",Accela powers thousands of services and millio...,Accela improves citizen engagement by making i...,101+,"Health/Healthcare, Housing, Public Safety",NaN,[],NaN,2014-09-16 16:30:49.172802


In [20]:
ny_companies_df = company_df.loc[company_df['state']=='NY']

In [21]:
ny_companies_df=ny_companies_df[['company_name','url', 'year_founded','description','full_time_employees','zip_code']]

geolocation

In [22]:
geolocator = Nominatim()
location = geolocator.geocode("19114")
print((location.latitude, location.longitude))

In [23]:
geolocator = Nominatim()
def zip_to_lat_long(zipcode):
    location = geolocator.geocode(zipcode)
    latitude=location.latitude
    longitude=location.longitude
    #return pd.Series({'latitude':latitude,'longitude':longitude})
    return latitude,longitude

In [24]:
ny_companies_df['latitude'],ny_companies_df['longitude'] = zip(*ny_companies_df.zip_code.map(zip_to_lat_long))

In [25]:
ny_companies_df.head()

,company_name,url,year_founded,description,full_time_employees,zip_code,latitude,longitude
10,Aidin,http://www.myAidin.com,2011,Aidin helps hospitals and their patients choos...,1-10,10013,25.033144,121.517527
12,Allianz,http://www.allianzusa.com,1896,Allianz Life Insurance Company is a provider o...,"10,001+",10019,45.381550,7.887893
25,AreaVibes Inc.,http://www.areavibes.com/,2009,AreaVibe works to provide you with everything ...,1-10,NaN,34.220389,70.380031
29,Asset4,asset4.com,2003,ASSET4 AG provides investment research informa...,51-200,10036,45.136780,7.762083
41,Be Informed,http://www.beinformed.com/BeInformed/website/e...,2006,"Be Informed is an internationally operating, i...",51-200,10022,44.849031,7.734883


In [26]:
ny_companies_df.to_csv('geo/companies_geo.csv',index=False)

# Restaurants

In [27]:
rest_geo_df=pd.read_csv('geo/NYC_Geocoded_Restaurant_Locations.csv')

In [28]:
rest_geo_df.head()

,CAMIS,Longitude,Latitude,Source,Dataset Date
0,41670166,-73.973873,40.686146,TIGER/ArcMap,5/15/2015
1,41670151,-73.981241,40.763977,TIGER/ArcMap,5/15/2015
2,41670042,-73.952571,40.586910,TIGER/ArcMap,5/15/2015
3,41670066,-73.990436,40.771542,TIGER/ArcMap,5/15/2015
4,41670188,-73.899113,40.825874,TIGER/ArcMap,5/15/2015


In [29]:
rest_name_df=pd.read_csv('geo/DOHMH_New_York_City_Restaurant_Inspection_Results.csv')

IOError: File geo/DOHMH_New_York_City_Restaurant_Inspection_Results.csv does not exist

In [ ]:
len(rest_name_df)

In [ ]:
rest_name_df.head()

In [ ]:
rest_name_df=rest_name_df[['DBA','SCORE','GRADE','CUISINE DESCRIPTION','PHONE','CAMIS']]

In [ ]:
rest_groups=rest_name_df.groupby(['CAMIS'])
rest_single=rest_groups.max()
rest_single=rest_single.reset_index()
rest_single.head()

In [ ]:
merged_res=pd.merge(rest_geo_df,rest_single,on='CAMIS',how='inner')

In [ ]:
merged_res.head()

In [ ]:
merged_res.to_csv('geo/res_merged.csv',index=False)